## Prepare train_data and test_data

In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from hypergbm import make_experiment
from hypernets.tabular.metrics import metric_to_scoring
from sklearn.metrics import get_scorer

In [2]:
X,y = datasets.load_breast_cancer(as_frame=True,return_X_y=True)
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.7,random_state=335)
train_data = pd.concat([X_train,y_train],axis=1)

# Add your own estimator(svm)

In [3]:
from hypergbm.sklearn.sklearn_ops import numeric_pipeline_simple,categorical_pipeline_simple
from hypergbm.estimators import HyperEstimator
from hypernets.pipeline.base import DataFrameMapper
from hypernets.core.ops import ModuleChoice, HyperInput
from hypernets.core.search_space import Choice, Int,Real
from hypernets.core.search_space import HyperSpace
from hypernets.tabular.column_selector import column_object
from hypernets.utils import const
from sklearn import svm

## Define SVMEstimator

In [4]:
class SVMEstimator(HyperEstimator):
    def __init__(self, fit_kwargs, C=1.0, kernel='rbf', gamma='auto',degree=3,random_state=666,probability=True,
                 decision_function_shape=None,space=None, name=None, **kwargs):
        if C is not None:
            kwargs['C'] = C
        if kernel is not None:
            kwargs['kernel'] = kernel
        if gamma is not None:
            kwargs['gamma'] = gamma
        if degree is not None:
            kwargs['degree'] = degree
        if random_state is not None:
            kwargs['random_state'] = random_state
        if decision_function_shape is not None:
            kwargs['decision_function_shape'] = decision_function_shape
        kwargs['probability'] = probability
        HyperEstimator.__init__(self, fit_kwargs, space, name, **kwargs)
    def _build_estimator(self, task, kwargs):
        if task == const.TASK_REGRESSION:
            hsvm = SVMRegressorWrapper(**kwargs)
        else:
            hsvm = SVMClassifierWrapper(**kwargs)
        hsvm.__dict__['task'] = task
        return hsvm
class SVMClassifierWrapper(svm.SVC):
    def fit(self, X, y=None, **kwargs):
        return super().fit(X, y)
class SVMRegressorWrapper(svm.SVC):
    def fit(self, X, y=None, **kwargs):
        return super().fit(X, y)

## Define search_space

In [5]:
def search_space():
    space = HyperSpace()
    with space.as_default():
        input = HyperInput(name='input1')
        num_pipeline = numeric_pipeline_simple()(input)
        cat_pipeline = categorical_pipeline_simple()(input)
        union_pipeline = DataFrameMapper(default=False, input_df=True, df_out=True,
                                         df_out_dtype_transforms=[(column_object, 'int')])([num_pipeline, cat_pipeline])
        svm_init_kwargs = {
            'C': Real(0.1,5,0.1),
            'kernel':Choice(['rbf','poly','sigmoid']),
            'degree':Int(1,5),
            'gamma':Real(0.0001,5,0.0002)
        }
        svm_est =SVMEstimator(fit_kwargs={},**svm_init_kwargs)
        ModuleChoice([svm_est], name='estimator_options')(union_pipeline)
        space.set_inputs(input)
    return space

In [6]:
experiment = make_experiment(train_data.copy(),target='target',test_data=X_test,
                             search_space=search_space)
estimator = experiment.run()

In [7]:
scorer = get_scorer(metric_to_scoring('precision',pos_label=1))
score = scorer(estimator, X_test, y_test)
score

0.9658119658119658

# Add SVM into GeneralSearchSpaceGenerator

In [8]:
from hypergbm.search_space import GeneralSearchSpaceGenerator
from hypergbm.estimators import XGBoostEstimator,LightGBMEstimator,CatBoostEstimator

class GeneralSearchSpaceGeneratorWithSVM(GeneralSearchSpaceGenerator):
    def __init__(self,enable_svm=True, **kwargs):
        super().__init__(**kwargs)
        self.enable_svm = enable_svm
    @property
    def default_svm_init_kwargs(self):
        return {
            'C': Real(0.1,5,0.1),
            'kernel':Choice(['rbf','poly','sigmoid']),
            'degree':Int(1,5),
            'gamma':Real(0.0001,5,0.0002)
        }
    @property
    def default_svm_fit_kwargs(self):
        return {}
    @property
    def estimators(self):
        r=super().estimators
        if self.enable_svm:
            r['svm'] = (SVMEstimator, self.default_svm_init_kwargs, self.default_svm_fit_kwargs)
        return r
    
generalSearchSpaceGeneratorWithSVM = GeneralSearchSpaceGeneratorWithSVM()

In [9]:
experiment = make_experiment(train_data.copy(),target='target',test_data=X_test,
                             search_space=generalSearchSpaceGeneratorWithSVM)
estimator = experiment.run()

In [10]:
scorer = get_scorer(metric_to_scoring('precision',pos_label=1))
score = scorer(estimator, X_test, y_test)
score

0.9652173913043478